<h1>Project: Family Income and Expenditure</h1>

<h2>Analyst: Serna Tricia and Palas Jamaica</h2>

---

In [109]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, r2_score

In [110]:
df = pd.read_csv("Family Income and Expenditure.csv")

df.head()

,Total Household Income,Region,Total Food Expenditure,Main Source of Income,Agricultural Household indicator,Bread and Cereals Expenditure,Total Rice Expenditure,Meat Expenditure,Total Fish and marine products Expenditure,Fruit Expenditure,...,Number of Refrigerator/Freezer,Number of Washing Machine,Number of Airconditioner,"Number of Car, Jeep, Van",Number of Landline/wireless telephones,Number of Cellular phone,Number of Personal Computer,Number of Stove with Oven/Gas Range,Number of Motorized Banca,Number of Motorcycle/Tricycle
0,480332,CAR,117848,Wage/Salaries,0,42140,38300,24676,16806,3325,...,1,1,0,0,0,2,1,0,0,1
1,198235,CAR,67766,Wage/Salaries,0,17329,13008,17434,11073,2035,...,0,1,0,0,0,3,1,0,0,2
2,82785,CAR,61609,Wage/Salaries,1,34182,32001,7783,2590,1730,...,0,0,0,0,0,0,0,0,0,0
3,107589,CAR,78189,Wage/Salaries,0,34030,28659,10914,10812,690,...,0,0,0,0,0,1,0,0,0,0
4,189322,CAR,94625,Wage/Salaries,0,34820,30167,18391,11309,1395,...,1,0,0,0,0,3,0,0,0,1


<h1>DESCRIPTIVE</h1>

In [127]:
# Filter Region IX
region9 = df[df["Region"] == "IX - Zasmboanga Peninsula"].copy()
region9.shape

(1788, 60)

<h2>This shows how many families are in the Region IX data and how many columns of information we have.</h2>

In [128]:
# See first rows of Region IX
region9.head()

,Total Household Income,Region,Total Food Expenditure,Main Source of Income,Agricultural Household indicator,Bread and Cereals Expenditure,Total Rice Expenditure,Meat Expenditure,Total Fish and marine products Expenditure,Fruit Expenditure,...,Number of Refrigerator/Freezer,Number of Washing Machine,Number of Airconditioner,"Number of Car, Jeep, Van",Number of Landline/wireless telephones,Number of Cellular phone,Number of Personal Computer,Number of Stove with Oven/Gas Range,Number of Motorized Banca,Number of Motorcycle/Tricycle
35187,192922,IX - Zasmboanga Peninsula,114258,Wage/Salaries,0,23791,14400,9607,10194,16586,...,1,0,0,0,0,3,0,0,0,0
35188,161843,IX - Zasmboanga Peninsula,78176,Wage/Salaries,0,30602,18750,5224,5684,4358,...,1,0,0,0,0,3,0,0,0,0
35189,535899,IX - Zasmboanga Peninsula,92464,Wage/Salaries,0,27200,20950,16848,12187,2225,...,1,0,0,0,0,4,0,0,0,2
35190,312579,IX - Zasmboanga Peninsula,133445,Enterpreneurial Activities,0,56237,41700,5242,7894,3109,...,1,0,0,0,0,3,0,0,0,1
35191,154715,IX - Zasmboanga Peninsula,39580,Other sources of Income,0,16592,13545,4711,5125,2245,...,0,0,0,0,0,3,1,0,0,0


<h2>Looking at the first rows shows what kind of information we have for each family, like income and spending..</h2>

In [129]:
# Summary for income and food spending
region9[["Total Household Income", "Total Food Expenditure"]].describe()

,Total Household Income,Total Food Expenditure
count,1.788000e+03,1788.000000
mean,1.910009e+05,69645.318233
std,2.382293e+05,44465.585388
min,1.973000e+04,5408.000000
25%,8.526350e+04,40500.250000
50%,1.265670e+05,59256.000000
75%,2.054888e+05,87221.000000
max,6.071030e+06,434881.000000


<h2>ShowThese numbers tell us the typical income and food spending and how much they vary between families..</h2>

In [130]:
# Food share of income
region9["Food_Percent"] = region9["Total Food Expenditure"] / region9["Total Household Income"]
region9["Food_Percent"].head()

35187    0.592250
35188    0.483036
35189    0.172540
35190    0.426916
35191    0.255825
Name: Food_Percent, dtype: float64

<h2>This tells how much of their money families spend on food. A bigger number means food takes more of their income.</h2>

In [131]:
# Share of families below median income
median_income = region9["Total Household Income"].median()
(region9["Total Household Income"] < median_income).mean()

np.float64(0.5)

<h2>This gives the portion of families that earn less than the middle family in the region.</h2>

In [132]:
# Count of types of toilet facilities
region9["Toilet Facilities"].value_counts(dropna=False)

Toilet Facilities
Water-sealed, sewer septic tank, used exclusively by household    1262
Water-sealed, sewer septic tank, shared with other household       138
Closed pit                                                         133
Open pit                                                            94
Water-sealed, other depository, used exclusively by household       66
NaN                                                                 43
Others                                                              27
Water-sealed, other depository, shared with other household         25
Name: count, dtype: int64

<h2>This shows how many families have different toilet types, which tells us about home sanitation.</h2>

In [133]:
# Electricity availability counts
region9["Electricity"].value_counts(dropna=False)

Electricity
1    1445
0     343
Name: count, dtype: int64

<h2>This shows how many families have electricity and how many do not, which relates to living comfort.</h2>

In [134]:
# Main source of water supply counts
region9["Main Source of Water Supply"].value_counts(dropna=False)

Main Source of Water Supply
Own use, faucet, community water system    591
Shared, tubed/piped deep well              285
Dug well                                   253
Shared, faucet, community water system     240
Protected spring, river, stream, etc       176
Unprotected spring, river, stream, etc      76
Peddler                                     70
Own use, tubed/piped deep well              43
Tubed/piped shallow well                    40
Lake, river, rain and others                 7
Others                                       7
Name: count, dtype: int64

<h2>This shows the common water sources families use, like deep wells, tap water, or other sources.</h2>

In [135]:
# Household size and children counts
region9[["Total Number of Family members", "Members with age less than 5 year old", "Members with age 5 - 17 years old"]].describe()

,Total Number of Family members,Members with age less than 5 year old,Members with age 5 - 17 years old
count,1788.000000,1788.000000,1788.000000
mean,4.552013,0.435682,1.411633
std,2.189429,0.708729,1.423096
min,1.000000,0.000000,0.000000
25%,3.000000,0.000000,0.000000
50%,4.000000,0.000000,1.000000
75%,6.000000,1.000000,2.000000
max,15.000000,5.000000,7.000000


<h2>This shows how many people live in a typical family and how many young children there are.</h2>

In [136]:
# Education level of household head
region9["Household Head Highest Grade Completed"].value_counts(dropna=False)

Household Head Highest Grade Completed
Elementary Graduate                                                                                                                                             324
High School Graduate                                                                                                                                            243
Grade 4                                                                                                                                                         157
Grade 5                                                                                                                                                         141
Second Year High School                                                                                                                                         120
Grade 3                                                                                                                                      

<h2>This lists the schooling levels of family heads, which helps understand job and income chances.</h2>

In [138]:
# Employment and business indicators (SAFE VERSION)

job_business_counts = region9["Household Head Job or Business Indicator"].value_counts(dropna=False)
employed_members_counts = region9["Total number of family members employed"].value_counts(dropna=False)

job_business_counts, employed_members_counts

(Household Head Job or Business Indicator
 With Job/Business    1541
 No Job/Business       247
 Name: count, dtype: int64,
 Total number of family members employed
 1    776
 0    502
 2    378
 3     88
 4     32
 5     10
 6      2
 Name: count, dtype: int64)

<h2>This shows how many people in each family have a job.
Some families have only one worker, which can make it hard to cover all expenses.
Other families have several working members, which helps them manage food, school, and daily needs more easily.</h2>

In [140]:
# Medical care spending summary
region9["Medical Care Expenditure"].describe()

count      1788.000000
mean       4165.670022
std       16456.561994
min           0.000000
25%         280.000000
50%         760.000000
75%        2139.250000
max      354230.000000
Name: Medical Care Expenditure, dtype: float64

<h2>This shows how much families spend on medical care on average and how this varies.</h2>

In [144]:
# One-step calculation: total expenditures and estimated savings
expenditure_cols = [
    'Total Food Expenditure', 'Bread and Cereals Expenditure',
    'Total Rice Expenditure', 'Meat Expenditure',
    'Total Fish and  marine products Expenditure',
    'Fruit Expenditure', 'Vegetables Expenditure',
    'Restaurant and hotels Expenditure',
    'Alcoholic Beverages Expenditure', 'Tobacco Expenditure',
    'Clothing, Footwear and Other Wear Expenditure',
    'Housing and water Expenditure', 'Imputed House Rental Value',
    'Medical Care Expenditure', 'Transportation Expenditure',
    'Communication Expenditure', 'Education Expenditure',
    'Miscellaneous Goods and Services Expenditure',
    'Special Occasions Expenditure', 'Crop Farming and Gardening expenses'
]

region9["Estimated_Savings"] = (
    region9["Total Household Income"] - region9[expenditure_cols].sum(axis=1)
)

region9["Estimated_Savings"].describe()

count    1.788000e+03
mean    -3.583276e+04
std      9.629667e+04
min     -4.380640e+05
25%     -7.320125e+04
50%     -4.408100e+04
75%     -1.777950e+04
max      1.659442e+06
Name: Estimated_Savings, dtype: float64

<h2>Most households in Region IX have negative estimated savings, meaning their expenses are higher than their income, which shows that many families in the region are experiencing financial strain.</h2>

<h1>DIAGNOSTIC</h2>

In [145]:
# Missing values per column
region9.isnull().sum().sort_values(ascending=False).head(20)

Household Head Occupation                        247
Household Head Class of Worker                   247
Toilet Facilities                                 43
Main Source of Income                              0
Agricultural Household indicator                   0
Region                                             0
Total Household Income                             0
Meat Expenditure                                   0
Total Fish and  marine products Expenditure        0
Fruit Expenditure                                  0
Vegetables Expenditure                             0
Restaurant and hotels Expenditure                  0
Alcoholic Beverages Expenditure                    0
Bread and Cereals Expenditure                      0
Total Food Expenditure                             0
Clothing, Footwear and Other Wear Expenditure      0
Housing and water Expenditure                      0
Medical Care Expenditure                           0
Imputed House Rental Value                    

<h2>This tells which pieces of information are missing the most and helps check data quality.</h2>

In [146]:
# Income skewness
region9["Total Household Income"].skew()

np.float64(10.509101926752688)

<h2>INSIGHT

This number shows that a few families earn much more than most, making the income numbers uneven.</h2>

In [147]:
# Count of very high income households (top 1%)
q99 = region9["Total Household Income"].quantile(0.99)
region9[region9["Total Household Income"] > q99].shape[0]

18

<h2>This counts how many families earn much more than most families in the region.</h2>

In [148]:
# Proportion with negative estimated savings
(region9["Estimated_Savings"] < 0).mean()

np.float64(0.8585011185682326)

<h2>This shows how many families appear to spend more than they earn, which points to financial strain.</h2>

In [151]:
# Food burden by income decile
region9["income_decile"] = pd.qcut(region9["Total Household Income"].rank(method="first"), 10, labels=False, duplicates='drop')
region9.groupby("income_decile")["Food_Percent"].median()

income_decile
0    0.487764
1    0.525236
2    0.502826
3    0.487798
4    0.507192
5    0.463273
6    0.470861
7    0.435479
8    0.340824
9    0.238337
Name: Food_Percent, dtype: float64

<h2>This shows how much of their income families in each income group spend on food. Lower groups spend more of their money on food.</h2>

In [152]:
# Electricity access by income decile (counts)
region9.groupby("income_decile")["Electricity"].value_counts(dropna=False)

income_decile  Electricity
0              0              102
               1               77
1              1               93
               0               86
2              1              123
               0               56
3              1              141
               0               37
4              1              157
               0               22
5              1              162
               0               17
6              1              169
               0                9
7              1              170
               0                9
8              1              176
               0                3
9              1              177
               0                2
Name: count, dtype: int64

<h2>This shows that richer groups have more access to electricity and poorer groups have less.</h2>

In [153]:
# Mean income by education level of household head
region9.groupby("Household Head Highest Grade Completed")["Total Household Income"].mean().sort_values()

Household Head Highest Grade Completed
Grade 6                                                                                                                                                          75608.000000
Preschool                                                                                                                                                        79259.500000
No Grade Completed                                                                                                                                               84764.681319
Grade 2                                                                                                                                                         103932.887500
Grade 1                                                                                                                                                         105305.119048
Grade 3                                                                                    

<h2>This shows that families headed by people with more schooling earn more money on average.</h2>

In [154]:
# Mean income by number of employed members
region9.groupby("Total number of family members employed")["Total Household Income"].mean().sort_index()

Total number of family members employed
0    144546.394422
1    167008.146907
2    245070.314815
3    277263.943182
4    565593.562500
5    320133.200000
6    506440.000000
Name: Total Household Income, dtype: float64

<h2>This shows that families with more working members usually earn more overall.</h2>

In [155]:
# Households with high medical spending share
region9["Medical_Catastrophic_Flag"] = region9["Medical Care Expenditure"] > 0.1 * region9["Total Household Income"]
region9["Medical_Catastrophic_Flag"].mean()

np.float64(0.030201342281879196)

<h2>This gives the share of families that spend a large part of their income on medical care, showing health-related financial pressure.</h2>

In [156]:
# Bedrooms per person (crowding)
region9["Bedrooms_per_person"] = region9["Number of bedrooms"].replace(0, np.nan) / region9["Total Number of Family members"].replace(0, np.nan)
region9["Bedrooms_per_person"].describe()

count    1649.000000
mean        0.519243
std         0.400761
min         0.083333
25%         0.250000
50%         0.400000
75%         0.666667
max         4.000000
Name: Bedrooms_per_person, dtype: float64

<h2>This shows how crowded homes are. Smaller numbers mean more people share fewer bedrooms.</h2>

In [158]:
# Share of agricultural households
region9["Agricultural Household indicator"].mean()

np.float64(0.5363534675615212)

<h2>This shows what portion of families do farming. More farming families means more rural livelihoods.</h2>

<h1>PREDICTIVE</h1>

In [176]:
df["Total_Expenditure_Sum"] = df[
    [
        "Total Food Expenditure",
        "Bread and Cereals Expenditure",
        "Total Rice Expenditure",
        "Meat Expenditure",
        "Total Fish and  marine products Expenditure",
        "Fruit Expenditure",
        "Vegetables Expenditure",
        "Restaurant and hotels Expenditure",
        "Alcoholic Beverages Expenditure",
        "Tobacco Expenditure",
        "Clothing, Footwear and Other Wear Expenditure",
        "Housing and water Expenditure",
        "Imputed House Rental Value",
        "Medical Care Expenditure",
        "Transportation Expenditure",
        "Communication Expenditure",
        "Education Expenditure",
        "Miscellaneous Goods and Services Expenditure",
        "Special Occasions Expenditure",
        "Crop Farming and Gardening expenses"
    ]
].sum(axis=1)


In [178]:
expenditure_cols = [
    'Total Food Expenditure', 
    'Bread and Cereals Expenditure',
    'Total Rice Expenditure',
    'Meat Expenditure',
    'Total Fish and  marine products Expenditure',
    'Fruit Expenditure',
    'Vegetables Expenditure',
    'Restaurant and hotels Expenditure',
    'Alcoholic Beverages Expenditure',
    'Tobacco Expenditure',
    'Clothing, Footwear and Other Wear Expenditure',
    'Housing and water Expenditure',
    'Imputed House Rental Value',
    'Medical Care Expenditure',
    'Transportation Expenditure',
    'Communication Expenditure',
    'Education Expenditure',
    'Miscellaneous Goods and Services Expenditure',
    'Special Occasions Expenditure',
    'Crop Farming and Gardening expenses'
]

df["Total_Expenditure_Sum"] = df[expenditure_cols].sum(axis=1)


In [182]:
# Recreate savings
df["Estimated_Savings"] = df["Total Household Income"] - df["Total_Expenditure_Sum"]

---

In [190]:
X = df[["Total_Expenditure_Sum"]]
y = df["Estimated_Savings"]

model1 = LinearRegression().fit(X, y)
pred1 = model1.predict(X)

print("R²:", r2_score(y, pred1))
print("MAE:", mean_absolute_error(y, pred1))

R²: 0.004121787555116163
MAE: 71541.52321033324


<h2>Insight: Higher total spending is generally associated with lower savings, so this model examines how strongly spending levels reduce a household’s ability to save..</h2>

In [191]:
X = df[["Total Number of Family members"]]
y = df["Total Household Income"]

model2 = LinearRegression().fit(X, y)
pred2 = model2.predict(X)

print("R²:", r2_score(y, pred2))
print("MAE:", mean_absolute_error(y, pred2))

R²: 0.021068260163353725
MAE: 158306.37051661566


<h2>Insight: This model evaluates whether larger households tend to earn more or less overall, revealing the income dynamics associated with family size.</h2>

In [193]:
X = df[["House Floor Area"]]
y = df["Total Household Income"]

model3 = LinearRegression().fit(X, y)
pred3 = model3.predict(X)

print("R²:", r2_score(y, pred3))
print("MAE:", mean_absolute_error(y, pred3))

R²: 0.1177923866855437
MAE: 145182.08932352733


<h2>Home size often reflects economic capacity; this model assesses whether larger living spaces correspond to higher household income..</h2>

In [181]:
X = df[["Total_Expenditure_Sum"]]
y = df["Estimated_Savings"]

model4 = LinearRegression().fit(X, y)
pred4 = model4.predict(X)

<h2>This checks how well spending predicts income. A higher R² means people who spend more also tend to earn more.</h2>

In [195]:
X = df[["Total Number of Family members"]]
y = df["Total Food Expenditure"]

model5 = LinearRegression().fit(X, y)
pred5 = model5.predict(X)

print("R²:", r2_score(y, pred5))
print("MAE:", mean_absolute_error(y, pred5))

R²: 0.17488076466071145
MAE: 33232.32609386424


<h2>Food costs typically increase as household size grows; this model quantifies how much additional food expenditure is associated with each extra family member.</h2>

In [196]:
X = df[["Total Household Income"]]
y = df["Communication Expenditure"]

model6 = LinearRegression().fit(X, y)
pred6 = model6.predict(X)

print("R²:", r2_score(y, pred6))
print("MAE:", mean_absolute_error(y, pred6))

R²: 0.5042990142050536
MAE: 2568.227678715073


<h2>This model explores whether households with higher income levels tend to spend more on communication-related services such as internet and mobile plans.</h2>

In [197]:
X = df[["Members with age 5 - 17 years old"]]
y = df["Education Expenditure"]

model7 = LinearRegression().fit(X, y)
pred7 = model7.predict(X)

print("R²:", r2_score(y, pred7))
print("MAE:", mean_absolute_error(y, pred7))

R²: 0.0009982914107256047
MAE: 10247.782473669362


<h2>As school-age members increase, education expenses are expected to rise; this model evaluates the strength of that relationship.</h2>

In [198]:
X = df[["Number of Car, Jeep, Van"]]
y = df["Transportation Expenditure"]

model8 = LinearRegression().fit(X, y)
pred8 = model8.predict(X)

print("R²:", r2_score(y, pred8))
print("MAE:", mean_absolute_error(y, pred8))

R²: 0.2578543665756303
MAE: 9054.613806489977


<h2>Owning more vehicles usually increases transport-related costs; this model measures how each additional vehicle affects transportation spending.</h2>

In [199]:
X = df[["House Age"]]
y = df["Housing and water Expenditure"]

model9 = LinearRegression().fit(X, y)
pred9 = model9.predict(X)

print("R²:", r2_score(y, pred9))
print("MAE:", mean_absolute_error(y, pred9))

R²: 0.022016611255993856
MAE: 27486.11537995072


<h2>Older houses may require more maintenance and repairs; this model analyzes whether aging structures lead to higher housing and water expenses.</h2>

In [200]:
X = df[["Household Head Age"]]
y = df["Medical Care Expenditure"]

model10 = LinearRegression().fit(X, y)
pred10 = model10.predict(X)

print("R²:", r2_score(y, pred10))
print("MAE:", mean_absolute_error(y, pred10))

R²: 0.011912912770233985
MAE: 9108.674898985406


<h2>This model examines whether older household heads are associated with increased medical expenditures, reflecting age-related health needs.</h2>

In [201]:
X = df[["Total Household Income"]]
y = df["Restaurant and hotels Expenditure"]

model11 = LinearRegression().fit(X, y)
pred11 = model11.predict(X)

print("R²:", r2_score(y, pred11))
print("MAE:", mean_absolute_error(y, pred11))

R²: 0.2601169748185189
MAE: 11580.691277469734


<h2>Higher-income households often have more discretionary spending; this model checks if increased income leads to more spending on dining and hotels.</h2>

In [202]:
X = df[["Total Number of Family members"]]
y = df["Clothing, Footwear and Other Wear Expenditure"]

model12 = LinearRegression().fit(X, y)
pred12 = model12.predict(X)

print("R²:", r2_score(y, pred12))
print("MAE:", mean_absolute_error(y, pred12))

R²: 0.010847432917060362
MAE: 4068.9748436355717


<h2>Clothing needs rise with more family members; this model evaluates how much clothing expenditure increases as household size grows.</h2>

In [203]:
X = df[["Total Household Income"]]
y = df["Tobacco Expenditure"]

model13 = LinearRegression().fit(X, y)
pred13 = model13.predict(X)

print("R²:", r2_score(y, pred13))
print("MAE:", mean_absolute_error(y, pred13))

R²: 0.004745957386474986
MAE: 2719.096514905169


<h2>This model investigates whether tobacco consumption is income-sensitive, examining if tobacco spending increases with higher earnings.</h2>

In [204]:
X = df[["Total Household Income"]]
y = df["Alcoholic Beverages Expenditure"]

model14 = LinearRegression().fit(X, y)
pred14 = model14.predict(X)

print("R²:", r2_score(y, pred14))
print("MAE:", mean_absolute_error(y, pred14))

R²: 0.023526624106149585
MAE: 1228.445199151878


<h2>Alcohol purchases may reflect lifestyle and disposable income; this model assesses whether alcohol spending rises with household income.</h2>

In [205]:
X = df[["Number of Television"]]
y = df["Estimated_Savings"]

model15 = LinearRegression().fit(X, y)
pred15 = model15.predict(X)

print("R²:", r2_score(y, pred15))
print("MAE:", mean_absolute_error(y, pred15))

R²: 0.014237257529769898
MAE: 71900.3144471688


<h2>Household assets can provide insights into lifestyle tendencies; this model tests whether households with more t`melevisions save more or less.</h2>

---

In [212]:
calabarzon = df[df["Region"] == "IVA - CALABARZON"]

# Create comparison table
comparison = pd.DataFrame({
    "Average Income": [
        region9["Total Household Income"].mean(),
        calabarzon["Total Household Income"].mean()
    ],
    "Average Expenditure": [
        region9["Total_Expenditure_Sum"].mean(),
        calabarzon["Total_Expenditure_Sum"].mean()
    ],
    "Average Savings": [
        region9["Estimated_Savings"].mean(),
        calabarzon["Estimated_Savings"].mean()
    ],
    "Average Family Size": [
        region9["Total Number of Family members"].mean(),
        calabarzon["Total Number of Family members"].mean()
    ],
    "Average Education Spending": [
        region9["Education Expenditure"].mean(),
        calabarzon["Education Expenditure"].mean()
    ],
    "Average Food Spending": [
        region9["Total Food Expenditure"].mean(),
        calabarzon["Total Food Expenditure"].mean()
    ]
}, index=["Region IX – Zamboanga Peninsula", "Region IV-A – CALABARZON"])

comparison

,Average Income,Average Expenditure,Average Savings,Average Family Size,Average Education Spending,Average Food Spending
Region IX – Zamboanga Peninsula,191000.908277,226833.663311,-35832.755034,4.552013,4458.274609,69645.318233
Region IV-A – CALABARZON,303360.536040,367406.432244,-64045.896204,4.451706,10028.401249,105333.949543


<h2>CALABARZON households generally have higher income, higher spending, and greater savings capacity compared to Region IX. Region IX families tend to allocate more of their budget to essential needs, showing signs of tighter financial conditions and lower disposable income. Overall, CALABARZON reflects a more stable life status, while Region IX shows more economic vulnerability.</h2>